In [6]:
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure,ColumnDataSource
from scipy.optimize import curve_fit
import numpy as np
from functions import *
output_notebook()

path = '../visualisation/data/'
year = '2019'
month = '08'
day='14'
file_nums = ['003']
data_files = load_data(year,month,day,file_nums,path)    

sources = []
for d in data_files:
    sources.append(ColumnDataSource(d))

Loading BokehJS ...

In [7]:
def damped_sine(t,A,f,phi,gamma):
    return A*np.sin(t*f+phi)*np.exp(-gamma*t)
def fit_damped_sine(data_file,freq_guess=1e-7):
    """
    given a data file will return the parameters of the fit
    """
    sine_max = np.max(data_file['f'])
    sine_min = np.min(data_file['f'])
    sine_ave = (sine_max + sine_min)/2

    amplitude_guess =  (sine_max - sine_min)/2
    phase_guess = 0.00
    decay_guess = 100  #1 micros
    frequency_guess= 0  #100 ns

    params = curve_fit(damped_sine, data_file['v0'],\
                       data_file['f'] - sine_ave,\
                       [amplitude_guess,frequency_guess,phase_guess,decay_guess])

    fitted_vals=[]
    t_max = np.max(data_file['v0'])
    ts = 1.1*np.linspace(0,t_max,1000)
    for t in ts:
        fitted_vals.append(damped_sine(t,*params[0]))
    fitted_vals = np.asarray(fitted_vals)
    
    return ts,fitted_vals + sine_ave,params[0]
def get_pi_time(A,f,phi,gamma):
    return ((np.pi/2) - phi)/f
def plot_pi(ts,fit,params):
    """
    will return coordinates to plot a line showing the pi pulse
    """
    zero_y = np.min(fit)
    max_y = np.max(fit)
    t_pi = get_pi_time(*params)
    return np.asarray([t_pi,t_pi]),np.asarray([zero_y,max_y])

In [8]:
p = figure(plot_width=800, plot_height=400,tooltips=TOOLTIPS)
for i,s in enumerate(sources):
    p.line(x='v0', y = 'f',source = s, line_width=1,color=plot_colors[i], legend='file {}'.format(file_nums[i]),line_alpha=0.2)
    ts,fit,params=fit_damped_sine(df)
    pi_time = get_pi_time(*params)
    xs,ys = plot_pi(ts,fit,params)

    p.line(ts,fit,color=plot_colors[i],legend='pi time {} ns'.format(round(pi_time*1e9,3)))
    p.line(xs,ys,color=plot_colors[i],legend='pi time {} ns'.format(round(pi_time*1e9,3)))

p.legend.click_policy = "hide"

show(p)

NameError: name 'df' is not defined

In [9]:
source = ColumnDataSource(df)
p00 = figure(tooltips=TOOLTIPS)
p00.line(x='v0', y='f', source=source)
show(p00)

NameError: name 'df' is not defined

In [10]:
df

NameError: name 'df' is not defined